# Twitter

Para manipulação de dados, é possível trabalhar com diversas fontes. Neste exemplo vamos ver como utilizar o twitter e trabalhar com os dados que são retornados por lá.

In [ ]:
#Instalação dos pacotes
install.packages("twitteR")
install.packages("gsubfn")
install.packages("wordcloud")
install.packages("NLP")
install.packages("tm")

In [ ]:
#Carregamento dos pacotes
library(data.table)
library('twitteR')
library(igraph)
library(gsubfn)
library(wordcloud)
library(NLP)
library(tm)

## Autenticando no Twitter

Gerar as API Keys pelo site https://apps.twitter.com/

https://apps.twitter.com/app/12987492/keys

In [ ]:
options(httr_oauth_cache=T)

consumer_key = " " #API Key
consumer_secret = " " #API Secret Key
access_key = " " #Access Token
access_secret = " " #Access Token Secret

setup_twitter_oauth(consumer_key, consumer_secret, access_key, access_secret)

Para validar se a autenticação está realmente funcionando, vamos retornar a *localização* e a *descrição* de um @ qualquer. Peguei o meu para garantir que existe.

In [ ]:
usuario <- "DiegoNogare"
perfil <- getUser(usuario)

location(perfil)

description(perfil)

## Retorno de dados

### Dados do perfil 

In [ ]:
seguidores <- perfil$getFollowers()
length(seguidores)
seguidores <- rbindlist(lapply(seguidores,as.data.frame))


In [ ]:
head(seguidores)

In [ ]:
quemSigo <- perfil$getFriends()
length(quemSigo)
quemSigo <- rbindlist(lapply(quemSigo,as.data.frame))


In [ ]:
head(quemSigo)

E para analisar os meus seguidores que tem mais seguidores?

In [ ]:

SeguidoresQueTemMaisSeguidores <- seguidores[ order(-seguidores$followersCount) ,]
top10Seguidores <- head(SeguidoresQueTemMaisSeguidores,10 )

top10Seguidores <- top10Seguidores [,c('name', 'followersCount', 'screenName', 'location', 'profileImageUrl')]

print(top10Seguidores)

E das pessoas que eu sigo, quais tem mais seguidores?

In [ ]:
SigoQueTemMaisSeguidores <- quemSigo[ order(-quemSigo$followersCount) ,]
top10quemSigo <- head(SigoQueTemMaisSeguidores,10 )

top10quemSigo <- top10quemSigo [,c('name', 'followersCount', 'screenName', 'location', 'profileImageUrl')]

print(top10quemSigo)

## Retorno de tweets 

Retonar os ultimos tweets das pessoas que me seguem

In [ ]:
tweets <- as.data.frame(matrix(c(NA,NA), nrow=1,ncol=2))

ultimosTweets <- 1
dez <- length(top10Seguidores$screenName)
i <- 1
while (i <= dez)
{
   tweets <- rbind(tweets, as.data.frame( matrix(c(top10Seguidores[i]$screenName,
      userTimeline(top10Seguidores[i]$screenName, n=ultimosTweets)[[1]]$text
      ) ,nrow=ultimosTweets, ncol=2, byrow=T) )
      )
  i <- i + 1
}
colnames(tweets) <- c("Perfil", "Post")
tweetsSeguidores <- tweets[!is.na(tweets$Perfil),]

View(tweetsSeguidores)

E também, os ultimos tweets das pessoas que eu sigo

In [ ]:
tweets <- as.data.frame(matrix(c(NA,NA), nrow=1,ncol=2))

ultimosTweets <- 1
dez <- length(top10quemSigo$screenName)
i <- 1
while (i <= dez)
{
  tweets <- rbind(tweets, as.data.frame( matrix(c(top10quemSigo[i]$screenName,
                                                  userTimeline(top10quemSigo[i]$screenName, n=ultimosTweets)[[1]]$text
  ) ,nrow=ultimosTweets, ncol=2, byrow=T) )
  )
  i <- i + 1
}
colnames(tweets) <- c("Perfil", "Post")
tweetsQuemSigo <- tweets[!is.na(tweets$Perfil),]

View(tweetsQuemSigo)

Seguidores com a mesma localização que a minha 

In [ ]:
SeguidoresNaMesmaLocalizacaoQueEu <- as.data.frame( seguidores[seguidores$location == location(perfil),] )
head(SeguidoresNaMesmaLocalizacaoQueEu$name,10)

Quem eu sigo que também está com a mesma localização

In [ ]:
QuemSigoNaMesmaLocalizacaoQueEu <- as.data.frame( quemSigo[quemSigo$location == location(perfil),] )
head(QuemSigoNaMesmaLocalizacaoQueEu$name,10)


## Buscas de Tweets


In [ ]:
#Função para limpar texto
CaracteresParaMudar <- list( 'S'='S', 's'='s', 'Z'='Z', 'z'='z', 'À'='A', 'Á'='A', 'Â'='A', 'Ã'='A', 'Ä'='A', 'Å'='A', 'Æ'='A', 'Ç'='C', 'È'='E', 'É'='E',
                             'Ê'='E', 'Ë'='E', 'Ì'='I', 'Í'='I', 'Î'='I', 'Ï'='I', 'Ñ'='N', 'Ò'='O', 'Ó'='O', 'Ô'='O', 'Õ'='O', 'Ö'='O', 'Ø'='O', 'Ù'='U',
                             'Ú'='U', 'Û'='U', 'Ü'='U', 'Ý'='Y', 'Þ'='B', 'ß'='Ss', 'à'='a', 'á'='a', 'â'='a', 'ã'='a', 'ä'='a', 'å'='a', 'æ'='a', 'ç'='c',
                             'è'='e', 'é'='e', 'ê'='e', 'ë'='e', 'ì'='i', 'í'='i', 'î'='i', 'ï'='i', 'ð'='o', 'ñ'='n', 'ò'='o', 'ó'='o', 'ô'='o', 'õ'='o',
                             'ö'='o', 'ø'='o', 'ù'='u', 'ú'='u', 'û'='u', 'ý'='y', 'ý'='y', 'þ'='b', 'ÿ'='y' )


LimparTexto <- function(texto){
  texto <- gsub("\\s?(f|ht)(tp)(s?)(://)([^\\.]*)[\\.|/](\\S*)", texto, replacement="")
  texto <- gsub("\n", texto, replacement=" ")
  texto <- gsub(pattern="[[:punct:]]", texto, replacement="")
  
  texto <- tolower(texto)
  texto <- gsubfn(paste(names(CaracteresParaMudar),collapse='|'), CaracteresParaMudar, texto)

  texto <- removeWords(texto, stopwords("portuguese"))    
  return(texto)
}

prepararNuvem <- function()
{
  docs <- Corpus(VectorSource(postsLimpos))
  dtm <- TermDocumentMatrix(docs)
  m <- as.matrix(dtm)
  v <- sort(rowSums(m),decreasing=TRUE)
  d <- data.frame(word = names(v),freq=v)
  return(d)
}

In [ ]:
#Busca por termo específico
#**************************
totalTweets = 1000
termo <- "#NFLnaESPN"
desde <- "2021-01-01"
idioma <- 'pt'

posts <- searchTwitter(termo, n=totalTweets, since=desde, lang = idioma) 
posts <- twListToDF(posts)

> Execute a leitura do arquivo **somente** se você não conseguiu recuperar os termos específicos que gostaria. Tem alguns exemplos que carreguei para hoje:
>
>    - Bolsonaro.txt
>    
>    - inteligenciaArtificial.csv
>    
>    - Lula.csv
>    
>    - tokyo2020.csv
>    
>    - NFLnaESPN.csv

In [ ]:
#Ler o arquivo (caso não consiga fazer a busca por termos específicos)
arquivo <- "inteligenciaArtificial.csv"
posts <- read.csv(arquivo, header=TRUE, sep=",")

head(posts)

In [ ]:
head(posts)

In [ ]:
postsLimpos <- as.vector( as.character( LimparTexto(posts$text) ) )

head(postsLimpos)

In [ ]:
d <- prepararNuvem()

wordcloud(words = d$word, freq = d$freq, min.freq = 1
          ,max.words=150
          ,random.order=FALSE
          ,colors=brewer.pal(8, "Dark2") )

In [ ]:
nomeArquivo <- "NuvemPalavras.png"

png(nomeArquivo, width=1024, height=1024, units="px", res=150)
    wordcloud(words = d$word, freq = d$freq, min.freq = 1
              ,max.words=150
              ,random.order=FALSE
              ,colors=brewer.pal(8, "Dark2") )
dev.off()

## Publicação no Twitter 

In [ ]:
updateStatus( paste("Ao procurar '", termo, "' no idioma '", toupper(idioma) , "' desde ", desde, " foi gerada esta nuvem. #LinguagemR #DataScience" , sep="") , mediaPath = nomeArquivo)